In [1]:
from sklearn.metrics import *
import joblib
from collections import defaultdict
from scipy.interpolate import interp1d
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np

from tensorflow.keras.layers import BatchNormalization,LSTM,GRU,Dropout,Dense, SimpleRNN
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
import warnings
import datetime
import csv
import os
import datetime
warnings.filterwarnings(action='ignore')
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

2023-01-31 17:27:45.352464: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


dfh : hvac data
dfs : sensor data

In [2]:
df_train = pd.read_feather('/home/deallab/harin/ECM/ecm_input.ftr')
df_train = df_train.set_index('timestamp_min')

In [3]:
df_train

,f3_hum,f10_hum,f3_temp,f10_temp,exthum,enhum,temp,entemp,extemp,F_SET_MODE_0,F_SET_MODE_1,F_SET_MODE_2,F_SET_MODE_3,F_SET_MODE_4,F_SET_MODE_5
timestamp_min,,,,,,,,,,,,,,,
2019-08-21 16:37:00,57.37,51.87,25.47,26.77,59.6,85.6,81.5,29.2,27.3,0,0,0,0,1,0
2019-08-21 16:38:00,57.37,51.87,25.47,26.77,59.4,85.4,107.6,29.2,27.1,0,0,0,0,1,0
2019-08-21 16:39:00,57.27,51.85,25.50,26.77,57.3,85.4,105.2,29.2,27.3,0,0,0,0,1,0
2019-08-21 16:40:00,57.27,51.87,25.60,26.77,58.3,85.6,82.5,29.2,27.5,0,0,0,0,1,0
2019-08-21 16:41:00,57.20,51.85,25.60,26.77,59.5,85.8,76.9,29.2,27.3,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-10-24 23:55:00,36.72,40.12,23.62,22.32,52.6,83.1,40.0,19.9,20.8,1,0,0,0,0,0
2019-10-24 23:56:00,36.72,40.10,23.62,22.42,52.8,82.8,40.0,19.9,20.8,1,0,0,0,0,0
2019-10-24 23:57:00,36.85,40.12,23.62,22.42,52.8,83.0,40.0,19.9,20.8,1,0,0,0,0,0


In [4]:
input_feature = ['f3_hum','f10_hum','f3_temp','f10_temp','exthum','enhum','temp','entemp','extemp','F_SET_MODE_0','F_SET_MODE_1','F_SET_MODE_2','F_SET_MODE_3','F_SET_MODE_4','F_SET_MODE_5']
output_feature = ['f3_hum','f10_hum','f3_temp','f10_temp','exthum','temp','extemp']

In [5]:
EPOCHS = 100 #int
steps_per_epoch = 200 #int
BATCH_SIZE = 32 #int
BUFFER_SIZE = 1000000 #int
PATIENCE = 5 #i

In [7]:
# 모델의 Timewindow별 학습하기 위한 데이터를 나눔


def train_model_data(df_s, Train_windows_size, Train_date ):
    
    df_train = df_s[((pd.to_datetime(df_s.index).date) >= (Train_date + datetime.timedelta(days=-Train_windows_size))) & 
           ((pd.to_datetime(df_s.index).date) < Train_date)]
    
    df_test = df_s[(pd.to_datetime(df_s.index).date) == (Train_date)]
    
    return df_train, df_test

In [8]:
def build_gru_1layer_model():
    model = Sequential()
    model.add(GRU(128,input_shape = (1,len(input_feature))))
    model.add(Dropout(0.30))
    #model.add(Dropout(0.30))
    #model.add(Dense(32, activation='relu'))
    #model.add(Dense(16, activation='relu'))
    model.add(Dense(len(output_feature),activation = 'relu'))
    optimizer = tf.keras.optimizers.Adam()
    model.compile(loss = "mae", optimizer = optimizer)#, metrics = ['mae', 'mse'])
    return model

In [9]:
def build_lstm_1layer_model():
    model = Sequential()
    #model.add(BatchNormalization(input_shape = (1,len(input_feature))))
    model.add(LSTM(128,input_shape = (1,len(input_feature))))
    model.add(Dropout(0.30))
    #model.add(Dropout(0.30))
    #model.add(Dense(32, activation='relu'))
    #model.add(Dense(16, activation='relu'))
    model.add(Dense(len(output_feature),activation = 'relu'))
    optimizer = tf.keras.optimizers.Adam()
    model.compile(loss = "mae", optimizer = optimizer)#, metrics = ['mae', 'mse'])
    return model

In [10]:
def build_simple_rnn_1layer_model():
    model = Sequential()
    model.add(SimpleRNN(128,input_shape = (1,len(input_feature))))
    model.add(Dropout(0.30))
    #model.add(Dropout(0.30))
    #model.add(Dense(32, activation='relu'))
    #model.add(Dense(16, activation='relu'))
    model.add(Dense(len(output_feature),activation = 'relu'))
    optimizer = tf.keras.optimizers.Adam()
    model.compile(loss = "mae", optimizer = optimizer)#, metrics = ['mae', 'mse'])
    return model

In [11]:
# 모델 학습 및 결과 함수

def train_models(model_name, df_pre, Shift_Target, df_test_pre):
    
   
    if model_name == "gru_1layer":
        model = build_gru_1layer_model()
    elif model_name == "lstm_1layer":
        model = build_lstm_1layer_model()
    elif model_name == "simpleRNN":
        model = build_simple_rnn_1layer_model()
        
    x_train = df_pre[input_feature][:-10].values
    x_train  = x_train.reshape(-1,1,len(input_feature))
    y_train = df_pre[output_feature][10:].values
    x_test = df_test_pre[input_feature][:-10].values
    x_test  = x_test.reshape(-1,1,len(input_feature))
    y_test = df_test_pre[output_feature][10:].values
    

    model.fit(    x = x_train, 
                  y= y_train,
                    epochs = EPOCHS,
                    validation_split = 0.1,
                    steps_per_epoch=steps_per_epoch,
                    validation_steps=steps_per_epoch,
                    verbose=1,
        #callbacks=[early_stopping])
                 )
    y_pred = model.predict(x_test)
    #x_train, y_train =  multivariate_data(df_pre[input_feature].reset_index(drop=True).values, df_pre[output_feature].reset_index(drop=True).values, 0, None, past_history, future_target)
    #x_test, y_test =  multivariate_data(df_test_pre[input_feature].reset_index(drop=True).values, df_test_pre[output_feature].reset_index(drop=True).values, 0, None, past_history, future_target)
    #x_train = x_train.reshape(-1,past_history*len(input_feature))
    #x_test = x_test.reshape(-1,past_history*len(input_feature))
   
   
    #model.fit(df_pre[:-Shift_Target], df_pre[Shift_Target:])
    
    
    
    mae = mean_absolute_error(y_pred[:,2], y_test[:,2])
    rmse = mean_squared_error(y_pred[:,2], y_test[:,2], squared=False)
    r2 = r2_score(y_pred[:,2], y_test[:,2])
    mape = mean_absolute_percentage_error(y_pred[:,2], y_test[:,2])
    
#     print(f"MAE : " , mae)
#     print("RMSE: ", rmse)
#     print("R2: ", r2)
#     print("MAPE: ", mape)
    
    h_mae = mean_absolute_error(y_pred[:,0], y_test[:,0])
    h_rmse = mean_squared_error(y_pred[:,0], y_test[:,0], squared=False)
    h_r2 = r2_score(y_pred[:,0], y_test[:,0])
    h_mape = mean_absolute_percentage_error(y_pred[:,0], y_test[:,0])
    
#     print(f"h_MAE : " , h_mae)
#     print("h_RMSE: ", h_rmse)
#     print("h_R2: ", h_r2)
#     print("h_MAPE: ", h_mape)
    
    return model, y_pred, [mae, rmse, r2, mape, h_mae, h_rmse, h_r2, h_mape] 

In [12]:
# 모델 결과 값 및 모델 저장
def model_save(Train_date, Train_windows_size, model_name, export_path, model, model_error ):
    
    model_filename = Train_date.strftime("%Y%m%d") + "w" + str(Train_windows_size) + "_" + model_name + ".h5"
    
    filepath = os.path.join(export_path,model_filename)
    
    # extra_trees_regressor 모델 값이 너무 커서 저장하지 않음.
    # 향후 풀어야되는 문제임
    #if not model_name == "extra_trees_regressor":
    if not os.path.exists(f"{export_path}"):
        os.makedirs(f"{export_path}", exist_ok=True)
    if not os.path.isfile(filepath):
        model.save(filepath)
    print(f"{model_filename} 에 DL 모델이 저장되었습니다.")



    #엑셀(csv) 저장
    f = open(csv_name,'a+', newline='')
    wr = csv.writer(f)
    wr.writerow([Train_date.strftime("%Y-%m-%d"), model_name, str(Train_windows_size), 
              model_error[0],model_error[1],model_error[2],model_error[3],
              model_error[4],model_error[5],model_error[6],model_error[7]])
    f.close()

In [ ]:
#모델 결과 값 저장
csv_name = "ecm_dl.csv"

Shift_Target = 10

#모델 실행
model_list = ["gru_1layer", "lstm_1layer",'simpleRNN']

# 실험 파라메터
# Train Windows size
# Train 시작 일자

#Train_windows_size_list = [1,7,14,21,30] #날짜

start_date = datetime.date(2019,8,23)
end_date = datetime.date(2019,10,25)
std  = datetime.date(2019,8,22)

# 날짜 array 생성
if (end_date-start_date).days == 0:
    end_date = start_date + datetime.timedelta(days=1)

Train_date_list = [start_date + datetime.timedelta(days=x) for x in range(0, (end_date-start_date).days)]

# 모델 학습

for Train_date in Train_date_list:
    if Train_date - std < datetime.timedelta(days=7) : Train_windows_size_list = [1]
    elif Train_date - std < datetime.timedelta(days=14) : Train_windows_size_list = [1,7]
    elif Train_date - std < datetime.timedelta(days=30) : Train_windows_size_list = [1,7,14]
    else : Train_windows_size_list = [1,7,14,30] #날짜
        
    export_path = os.path.join('/raid/deallab/ecm', "model_result_dl")
    #export_path = os.path.join(os.getcwd(), "input wind")
    export_path = os.path.join(export_path, Train_date.strftime("%Y-%m-%d"))

    print(export_path)
    
    for Train_windows_size in Train_windows_size_list:

        df_pre, df_test_pre = train_model_data(df_train, Train_windows_size, Train_date )

        for m_name in model_list:

            model, y_pred, model_error  = train_models(m_name, df_pre, Shift_Target, df_test_pre)
            model_save(Train_date, Train_windows_size, m_name, export_path, model, model_error)

/raid/deallab/ecm/model_result_dl/2019-08-23


2023-01-31 17:29:00.633510: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2023-01-31 17:29:00.685837: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:08:00.0 name: Tesla V100-DGXS-32GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 31.75GiB deviceMemoryBandwidth: 836.37GiB/s
2023-01-31 17:29:00.685890: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2023-01-31 17:29:00.688347: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2023-01-31 17:29:00.690313: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcufft.so.10
2023-01-31 17:29:00.690667: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcurand.so

Epoch 1/100


2023-01-31 17:29:03.759028: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2023-01-31 17:29:04.019594: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudnn.so.7


200/200 [==============================] - 1s 7ms/step - loss: 35.5213 - val_loss: 23.2826
Epoch 2/100
200/200 [==============================] - 1s 3ms/step - loss: 17.9694
Epoch 3/100
200/200 [==============================] - 1s 4ms/step - loss: 8.5576
Epoch 4/100
200/200 [==============================] - 1s 3ms/step - loss: 7.0870
Epoch 5/100
200/200 [==============================] - 1s 3ms/step - loss: 6.4376
Epoch 6/100
200/200 [==============================] - 1s 3ms/step - loss: 5.9304
Epoch 7/100
200/200 [==============================] - 1s 3ms/step - loss: 5.6972
Epoch 8/100
200/200 [==============================] - 1s 3ms/step - loss: 5.6992
Epoch 9/100
200/200 [==============================] - 1s 3ms/step - loss: 5.5740
Epoch 10/100
200/200 [==============================] - 1s 3ms/step - loss: 5.4337
Epoch 11/100
200/200 [==============================] - 1s 3ms/step - loss: 5.3764
Epoch 12/100
200/200 [==============================] - 1s 3ms/step - loss: 5.2110
Epo

...

  0/200 [..............................] - 0s 0s/step - loss: 1.9925
20190826w1_simpleRNN.h5 에 DL 모델이 저장되었습니다.
/raid/deallab/ecm/model_result_dl/2019-08-27
Epoch 1/100
200/200 [==============================] - 1s 5ms/step - loss: 30.1173 - val_loss: 20.1795
Epoch 2/100
200/200 [==============================] - 1s 3ms/step - loss: 14.5858
Epoch 3/100
200/200 [==============================] - 1s 3ms/step - loss: 6.6929
Epoch 4/100
200/200 [==============================] - 1s 3ms/step - loss: 3.5880
Epoch 5/100
200/200 [==============================] - 1s 3ms/step - loss: 2.2238
Epoch 6/100
200/200 [==============================] - 1s 3ms/step - loss: 2.2222
Epoch 7/100
200/200 [==============================] - 1s 3ms/step - loss: 2.1074
Epoch 8/100
200/200 [==============================] - 1s 3ms/step - loss: 2.1750
Epoch 9/100
200/200 [==============================] - 1s 3ms/step - loss: 2.1858
Epoch 10/100
200/200 [==============================] - 1s 3ms/step - loss: 2.1536
E

  0/200 [..............................] - 0s 0s/step - loss: 1.9871
20190827w1_gru_1layer.h5 에 DL 모델이 저장되었습니다.
Epoch 1/100
200/200 [==============================] - 1s 7ms/step - loss: 33.0856 - val_loss: 25.8874
Epoch 2/100
200/200 [==============================] - 1s 4ms/step - loss: 20.4876
Epoch 3/100
200/200 [==============================] - 1s 4ms/step - loss: 12.2214
Epoch 4/100
200/200 [==============================] - 1s 4ms/step - loss: 7.0754
Epoch 5/100
200/200 [==============================] - 1s 4ms/step - loss: 4.7908
Epoch 6/100
200/200 [==============================] - 1s 3ms/step - loss: 2.8179
Epoch 7/100
200/200 [==============================] - 1s 3ms/step - loss: 2.2725
Epoch 8/100
200/200 [==============================] - 1s 3ms/step - loss: 2.2287
Epoch 9/100
200/200 [==============================] - 1s 3ms/step - loss: 2.2625
Epoch 10/100
200/200 [==============================] - 1s 3ms/step - loss: 2.3078
Epoch 11/100
200/200 [======================

  0/200 [..............................] - 0s 0s/step - loss: 2.0991
20190827w1_lstm_1layer.h5 에 DL 모델이 저장되었습니다.
Epoch 1/100
200/200 [==============================] - 1s 5ms/step - loss: 27.9036 - val_loss: 17.2856
Epoch 2/100
200/200 [==============================] - 1s 3ms/step - loss: 11.2851
Epoch 3/100
200/200 [==============================] - 1s 3ms/step - loss: 4.9137
Epoch 4/100
200/200 [==============================] - 1s 3ms/step - loss: 2.2192
Epoch 5/100
200/200 [==============================] - 1s 3ms/step - loss: 2.0304
Epoch 6/100
200/200 [==============================] - 1s 3ms/step - loss: 2.0169
Epoch 7/100
200/200 [==============================] - 1s 3ms/step - loss: 1.9875
Epoch 8/100
200/200 [==============================] - 1s 3ms/step - loss: 2.0182
Epoch 9/100
200/200 [==============================] - 1s 3ms/step - loss: 1.8847
Epoch 10/100
200/200 [==============================] - 1s 3ms/step - loss: 1.9613
Epoch 11/100
200/200 [======================

  0/200 [..............................] - 0s 0s/step - loss: 1.7344
20190827w1_simpleRNN.h5 에 DL 모델이 저장되었습니다.
/raid/deallab/ecm/model_result_dl/2019-08-28
Epoch 1/100
200/200 [==============================] - 1s 6ms/step - loss: 32.3731 - val_loss: 21.8994
Epoch 2/100
200/200 [==============================] - 1s 3ms/step - loss: 15.1080
Epoch 3/100
200/200 [==============================] - 1s 3ms/step - loss: 8.5690
Epoch 4/100
200/200 [==============================] - 1s 3ms/step - loss: 7.7495
Epoch 5/100
200/200 [==============================] - 1s 3ms/step - loss: 7.2088
Epoch 6/100
200/200 [==============================] - 1s 3ms/step - loss: 6.8145
Epoch 7/100
200/200 [==============================] - 1s 3ms/step - loss: 6.4974
Epoch 8/100
200/200 [==============================] - 1s 3ms/step - loss: 6.0164
Epoch 9/100
200/200 [==============================] - 1s 3ms/step - loss: 5.5031
Epoch 10/100
200/200 [==============================] - 1s 3ms/step - loss: 5.0874
E

  0/200 [..............................] - 0s 0s/step - loss: 3.2694
20190828w1_gru_1layer.h5 에 DL 모델이 저장되었습니다.
Epoch 1/100
200/200 [==============================] - 2s 8ms/step - loss: 35.1684 - val_loss: 28.0319
Epoch 2/100
200/200 [==============================] - 1s 3ms/step - loss: 21.4642
Epoch 3/100
200/200 [==============================] - 1s 3ms/step - loss: 12.7378
Epoch 4/100
200/200 [==============================] - 1s 3ms/step - loss: 8.5501
Epoch 5/100
200/200 [==============================] - 1s 3ms/step - loss: 7.6441
Epoch 6/100
200/200 [==============================] - 1s 3ms/step - loss: 7.1456
Epoch 7/100
200/200 [==============================] - 1s 3ms/step - loss: 6.6699
Epoch 8/100
200/200 [==============================] - 1s 3ms/step - loss: 6.2872
Epoch 9/100
200/200 [==============================] - 1s 3ms/step - loss: 5.9422
Epoch 10/100
200/200 [==============================] - 1s 3ms/step - loss: 5.5523
Epoch 11/100
200/200 [======================